<h1 style="background-color:#1c6ce6;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
🔀 תהליך הסבת טבלת סדר פעולות לבנק"ל מודרני 🔀 

In [ ]:
print('10. Sequence Actions Table:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import geopandas as gpd
import datetime as dt
import subprocess

In [ ]:
print('  10.a Reading inputs')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
RawData_folder = r"\\mapi_shares\MNCDB\צוות מידע\RawData"

today =  dt.date.today()
date_name =  str(today.year) + str(today.month) + str(today.day)
product_path = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים/' + date_name + '_ncdb.gdb'

<h1 style="text-align:right;font-size:100%">
שכבת גבולות תהליכי קדסטר מודרניים

In [ ]:
CPB_layer = gpd.read_file(product_path, layer = 'CadasterProcessBorders',
                          include_fields = ['CPBUniqueID', 'ProcessName', 'ProcessType'],
                          ignore_geometry = True)

CPB_layer = CPB_layer.astype({'CPBUniqueID' : int, 'ProcessName' : str})

CPB_tatag = CPB_layer[CPB_layer['ProcessType'] == 4]

<h1 style="text-align:right;font-size:150%">
:נתונים לפעולות של תצ"ר

<h1 style="text-align:right;font-size:100%">
טבלת סדר פעולות תצ"ר קיימת

In [ ]:
action_parcel_cols = ['GUSH_NUM', 'GUSH_SUFFIX', 'ACTION_NUM', 'LINE', 'TALAR_ID',
                      'FROM_PARCEL_TMP','FROM_PARCEL_FINAL', 'TO_GUSH_NUM', 'TO_GUSH_SUFFIX', 'TO_PARCEL_TMP', 'TO_PARCEL_FINAL']

action_parcel = pd.read_csv(RawData_folder + '/Oracle/action_parcel.csv', usecols = action_parcel_cols)

<h1 style="text-align:right;font-size:100%">
טבלת עדכון חלקות תצ"ר קיימת

In [ ]:
revision_action_cols = ['ACTION_TYPE', 'TALAR_ID', 'ACTION_NUM', 'ACTION_TYPE']

revision_action = pd.read_csv(RawData_folder + '/Oracle/REV_ACTION.csv', usecols = revision_action_cols)

<h1 style="text-align:right;font-size:100%"> 
טבלת פעולות של תצ"ר כרסטת

In [ ]:
talar_card = pd.read_csv(RawData_folder + '/Oracle/TALAR_ACTIONS.csv')

<h1 style="text-align:right;font-size:150%">
:נתונים לפעולות של תת"ג

In [ ]:
parcel_inprocess_cols = ['GUSHNUM', 'GUSHSUFFIX', 'PARCELNAME', 'TALAR_NUM', 'TALAR_YEAR', 'CADASTER_PROCESS']
parcel_inprocess = pd.read_csv(RawData_folder + '/Oracle/PARCELS_INPROCESS.csv', usecols = parcel_inprocess_cols, encoding = 'cp1256')

parcel_inprocess_tatag = parcel_inprocess[parcel_inprocess['CADASTER_PROCESS'] == 12]
parcel_inprocess_tatag.drop(columns = 'CADASTER_PROCESS', inplace=True)
parcel_inprocess_tatag.rename(columns={'TALAR_NUM' : 'TATAG_NUM',
                                       'TALAR_YEAR' : 'TATAG_YEAR'}, inplace=True)

In [ ]:
tatag_table_cols = ['TATAG_NUM', 'TATAG_YEAR', 'TATAG_STATUS']
tatag_table = pd.read_csv(RawData_folder + '/Oracle/TATAG.csv', usecols = tatag_table_cols)

<h1 style="text-align:right;font-size:150%">
:'נתונים לפעולות תיקוני 97 ב

In [ ]:
tikun_hesder_97b_cols = ['MISPAR_PNIA', 'GUSH_NUM', 'GUSH_SUFFIX', 'PARCELS', 'CODE_STEP']

tikun_hesder_97b = pd.read_csv(RawData_folder + '/Oracle/TIKUN_HESDER_97B.csv', usecols = tikun_hesder_97b_cols)

<h1 style="text-align:right;font-size:150%">
נתונים לפעולות פסדקי דין

<h1 style="text-align:right;font-size:100%">
טבלת פסק דין

In [ ]:
judgements_cols = ['PSAK_DIN_ID', 'STATUS', 'GUSH_NUM', 'GUSH_SUFFIX']

judgements = pd.read_csv(RawData_folder + '/Oracle/psak_din.csv', usecols = judgements_cols, encoding = 'cp1255')

<h1 style="text-align:right;font-size:100%">
טבלת חלקות פסקי דין

In [ ]:
judgements_parcels_cols = ['PSAK_DIN_ID', 'PARCEL', 'TYPE']

judgements_parcels = pd.read_csv(RawData_folder + '/Oracle/psak_din_parcels.csv', usecols = judgements_parcels_cols)

In [ ]:
del [RawData_folder, product_path, revision_action_cols, action_parcel_cols, parcel_inprocess_cols, parcel_inprocess, tatag_table_cols, CPB_layer, tikun_hesder_97b_cols, judgements_cols, judgements_parcels_cols]

In [ ]:
print('  10.b Cleaning data')

In [ ]:
print('  10.c Data conversion')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

<h1 style="text-align:right;font-size:150%">
:עיבוד נתוני תצ"ר

<h1 style="text-align:right;font-size:100%">
הוספת 1,000,000 למספר מזהה של כל התצ"ר לפי סכימה חדשה

In [ ]:
action_parcel['TALAR_ID']   = action_parcel['TALAR_ID'] + 1000000
revision_action['TALAR_ID'] = revision_action['TALAR_ID'] + 1000000
talar_card['TALARID']       = talar_card['TALARID'] + 1000000

<h1 style="text-align:right;font-size:100%">
הוספת שדה של סוג פעולה אל טבלת סדר פעולות תצ"ר על ידי מיזוג

In [ ]:
tazar_sequence = action_parcel.merge(revision_action, on=['TALAR_ID', 'ACTION_NUM'], how = 'left')
tazar_sequence = tazar_sequence.astype('Int64')

<h1 style="text-align:right;font-size:100%">
שינוי שמות של שדות
<h1 style="text-align:right;font-size:100%">
הוספת רשומות של תצ"ר כרטסת לטבלת סדר פעולות תצ"ר
<h1 style="text-align:right;font-size:100%">
הוספת שדה של סוג התהליך עם הערך 1 (תצ"ר)

In [ ]:
tazar_sequence.rename(columns = {'TALAR_ID'          : 'CPBUniqueID',
                                 'GUSH_NUM'          : 'BlockNumber',
                                 'GUSH_SUFFIX'       : 'SubBlockNumber',
                                 'ACTION_NUM'        : 'ActionNumber',
                                 'ACTION_TYPE'       : 'ActionType',
                                 'LINE'              : 'LineNumber',
                                 'FROM_PARCEL_TMP'   : 'FromParcelTemp',
                                 'FROM_PARCEL_FINAL' : 'FromParcelFinal',
                                 'TO_GUSH_NUM'       : 'ToBlockNumber',
                                 'TO_GUSH_SUFFIX'    : 'ToSubBlockNumber',
                                 'TO_PARCEL_TMP'     : 'ToParcelTemp',
                                 'TO_PARCEL_FINAL'   : 'ToParcelFinal'},
                      inplace = True)


talar_card.rename(columns = {'TALARID'             : 'CPBUniqueID',
                             'BLOCKNUMBER'         : 'BlockNumber',
                             'BLOCKSUFFIXNUMBER'   : 'SubBlockNumber',
                             'ACTIONNUMBER'        : 'ActionNumber',
                             'ACTIONTYPE'          : 'ActionType',
                             'LINENUMBER'          : 'LineNumber',
                             'FROMPARCELTEMP'      : 'FromParcelTemp',
                             'FROMPARCELFINAL'     : 'FromParcelFinal',
                             'TOBLOCKNUMBER'       : 'ToBlockNumber',
                             'TOBLOCKSUFFIXNUMBER' : 'ToSubBlockNumber',
                             'TOPARCELTEMP'        : 'ToParcelTemp',
                             'TOPARCELFINAL'       : 'ToParcelFinal'},
                      inplace = True)


tazar_sequence = tazar_sequence.append(talar_card)

tazar_sequence['ProcessType'] = 1

In [ ]:
del [action_parcel, revision_action, talar_card]

<h1 style="text-align:right;font-size:150%">
:עיבוד נתוני תת"ג

<h1 style="text-align:right;font-size:100%">
סינון רשומות מטבלת תת"ג לסטאטוס מספר 2 (מאושר)
<h1 style="text-align:right;font-size:100%">
יצירת שדה מקשר לפי שם מפה לצורך מיזוג בהמשך
<h1 style="text-align:right;font-size:100%">
הסרת שדות לא רלונטיים

In [ ]:
tatag_table = tatag_table[tatag_table['TATAG_STATUS'] == 2]

parcel_inprocess_tatag[['TATAG_NUM', 'TATAG_YEAR']] = parcel_inprocess_tatag[['TATAG_NUM', 'TATAG_YEAR']].astype(int)
tatag_table[['TATAG_NUM', 'TATAG_YEAR']] = tatag_table[['TATAG_NUM', 'TATAG_YEAR']].astype(int)

parcel_inprocess_tatag['ProcessName'] = parcel_inprocess_tatag['TATAG_NUM'].astype(str) + '/' + parcel_inprocess_tatag['TATAG_YEAR'].astype(str)
tatag_table['ProcessName'] = tatag_table['TATAG_NUM'].astype(str) + '/' + tatag_table['TATAG_YEAR'].astype(str)

parcel_inprocess_tatag.drop(columns = ['TATAG_NUM', 'TATAG_YEAR'], inplace=True)
tatag_table.drop(columns = ['TATAG_NUM', 'TATAG_YEAR'], inplace=True)

<h1 style="text-align:right;font-size:100%">
פירוק שדה שם חלקה לשני שדות: סטאטוס חלקה ומספר חלקה
<h1 style="text-align:right;font-size:100%">
מילוי ערכים חסרים במפר חלקה

In [ ]:
parcel_inprocess_tatag[['ParcelStatus', 'ParcelNumber']] = parcel_inprocess_tatag['PARCELNAME'].str.extract('([a-zA-Z]+)([^a-zA-Z]+)', expand=True)
parcel_inprocess_tatag.drop(columns='PARCELNAME', inplace=True)
parcel_inprocess_tatag['ParcelNumber'] = parcel_inprocess_tatag['ParcelNumber'].fillna(0)
parcel_inprocess_tatag['ParcelNumber'] = parcel_inprocess_tatag['ParcelNumber'].astype(int)

<h1 style="text-align:right;font-size:100%">
מיזוג טבלאות

In [ ]:
tatag_sequence = parcel_inprocess_tatag.merge(tatag_table, on='ProcessName', how='left')

del [parcel_inprocess_tatag, tatag_table]

<h1 style="text-align:right;font-size:100%">
הגדרת ערכי שדות של קישורי חלקות

In [ ]:
tatag_sequence['FromParcelFinal'] = np.where(tatag_sequence['ParcelStatus']=='F',
                                             tatag_sequence['ParcelNumber'],
                                             None)


tatag_sequence['ToParcelTemp'] = np.where(tatag_sequence['ParcelStatus']=='T',
                                          tatag_sequence['ParcelNumber'].astype('Int64'),
                                          None)


tatag_sequence['ToParcelFinal'] = np.where(tatag_sequence['ParcelStatus']=='F',
                                           tatag_sequence['ParcelNumber'].astype('Int64'),
                                            None)                                          # equals to 'FromParcelFinal'


tatag_sequence.drop(columns = ['ParcelStatus', 'ParcelNumber', 'TATAG_STATUS'], inplace=True)

<h1 style="text-align:right;font-size:100%">
הגדרת שדות לטבלה

In [ ]:
tatag_sequence = tatag_sequence.merge(CPB_tatag, on = 'ProcessName', how = 'left')
tatag_sequence.drop(columns='ProcessName', inplace=True)

del CPB_tatag


tatag_sequence.rename(columns = {'GUSHNUM'    : 'BlockNumber',
                                 'GUSHSUFFIX' : 'SubBlockNumber'},
                      inplace = True)


tatag_sequence['ProcessType'] = 4


tatag_sequence['ActionNumber'] = None


tatag_sequence['ActionType'] = 4


tatag_sequence['LineNumber'] = None


tatag_sequence['FromParcelTemp'] = None


tatag_sequence['ToBlockNumber'] = None

tatag_sequence['ToSubBlockNumber'] = None

<h1 style="text-align:right;font-size:150%">
:'עיבוד נתוני תיקון 97 ב

<h1 style="text-align:right;font-size:100%">
סינון רשומות של 97ב' לפי סטאטוס

In [ ]:
tikun_hesder_97b = tikun_hesder_97b[tikun_hesder_97b['CODE_STEP'].isin([5, 6])]

<h1 style="text-align:right;font-size:100%">
הוספת 5,000,000 למספרי מזהה של תהליך
<h1 style="text-align:right;font-size:100%">
פיזור הטבלה לפי חלקות ולא לפי מספר פניה
<h1 style="text-align:right;font-size:100%">
הגדרת שדה סוג תהליך לתיקון 97 ב
<h1 style="text-align:right;font-size:100%">
הגדרת שדות אחרים

In [ ]:
tikun_hesder_97b['CPBUniqueID'] = tikun_hesder_97b['MISPAR_PNIA'] + 6000000


tikun_hesder_97b = tikun_hesder_97b.assign(PARCELS = tikun_hesder_97b.PARCELS.str.split(',')).explode('PARCELS').reset_index(drop=True)
tikun_hesder_97b['PARCELS'] = tikun_hesder_97b['PARCELS'].astype(int)
tikun_hesder_97b.sort_values(['MISPAR_PNIA', 'PARCELS'], inplace=True)


tikun_hesder_97b['ProcessType'] = 6


tikun_hesder_97b['BlockNumber'] = tikun_hesder_97b['GUSH_NUM'].astype(int)


tikun_hesder_97b['SubBlockNumber'] = tikun_hesder_97b['GUSH_SUFFIX'].astype(int)


tikun_hesder_97b['ActionNumber'] = np.nan


tikun_hesder_97b['ActionType'] = 4


tikun_hesder_97b['LineNumber'] = np.nan


tikun_hesder_97b['FromParcelTemp'] = np.nan


tikun_hesder_97b['FromParcelFinal'] = tikun_hesder_97b['PARCELS']


tikun_hesder_97b['ToBlockNumber'] = np.nan


tikun_hesder_97b['ToSubBlockNumber'] = np.nan


tikun_hesder_97b['ToParcelTemp'] = np.nan


tikun_hesder_97b['ToParcelFinal'] = tikun_hesder_97b['PARCELS']



tikun_hesder_97b.sort_values(['CPBUniqueID', 'FromParcelFinal'], inplace=True)
tikun_hesder_97b.drop(columns=['MISPAR_PNIA', 'GUSH_NUM', 'GUSH_SUFFIX', 'PARCELS', 'CODE_STEP'], inplace=True)

In [ ]:
ammendment_97b_sequence = tikun_hesder_97b.drop_duplicates(['CPBUniqueID', 'FromParcelFinal'], keep='last')

del tikun_hesder_97b

<h1 style="text-align:right;font-size:150%">
:נתוני פסקי דין
<h1 style="text-align:right;font-size:100%">    
סינון פסקי דין לפי סטאטוס 1 ו-3
<h1 style="text-align:right;font-size:100%">
מיזוג פסקי דין עם חלקות של פסקי דין
<h1 style="text-align:right;font-size:100%">
הגדרת ואיכלוס שדות

In [ ]:
judgements = judgements[judgements['STATUS'].isin([1,3])]

In [ ]:
judgements_sequence = judgements.merge(judgements_parcels, on='PSAK_DIN_ID', how='left').astype('Int32')
judgements_sequence.dropna(subset=['TYPE'], inplace=True)
judgements_sequence.reset_index(inplace=True, drop=True)

del judgements

In [ ]:
judgements_sequence['FromParcelFinal'] = np.where(judgements_sequence['TYPE'] == 0,
                                                  judgements_sequence['PARCEL'],
                                                  None)


judgements_sequence['ToParcelFinal'] = np.where(judgements_sequence['TYPE'] == 1,
                                                judgements_sequence['PARCEL'],
                                                None)


judgements_sequence['CPBUniqueID'] = judgements_sequence['PSAK_DIN_ID'] + 4000000


judgements_sequence['ProcessType'] = 3


judgements_sequence['BlockNumber'] = judgements_sequence['GUSH_NUM']


judgements_sequence['SubBlockNumber'] = judgements_sequence['GUSH_SUFFIX'].fillna(0)


judgements_sequence['ActionNumber'] = None


judgements_sequence['ActionType'] = 0


judgements_sequence['LineNumber'] = None


judgements_sequence['FromParcelTemp'] = None


judgements_sequence['ToBlockNumber'] = None


judgements_sequence['ToSubBlockNumber'] = None


judgements_sequence['ToParcelTemp'] = None

In [ ]:
judgements_sequence.drop(columns=['PSAK_DIN_ID', 'GUSH_NUM', 'GUSH_SUFFIX', 'STATUS', 'PARCEL', 'TYPE'], inplace=True)

## איחוד תוצאות

In [ ]:
print('  10.d Defining new Layer')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨
<h1 style="text-align:right;font-size:100%">
איחוד של 4 טבלאות פעולה לטבלת סדר פעולות אחת

In [ ]:
sequences_list = [tazar_sequence, tatag_sequence, ammendment_97b_sequence, judgements_sequence]

for table in sequences_list:
    table = table.astype('Int64')

SequenceActionsTable = tazar_sequence.append(tatag_sequence)\
                                         .append(ammendment_97b_sequence)\
                                             .append(judgements_sequence)



del [tazar_sequence, tatag_sequence, ammendment_97b_sequence, judgements_sequence]


SequenceActionsTable.dropna(subset=['CPBUniqueID'], inplace=True)

SequenceActionsTable = SequenceActionsTable.astype('Int64')

In [ ]:
print('  10.e Exporting results to product folder')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾
<h1 style="text-align:right; font-size:100%;"> 
<h1 style="text-align:right; font-size:100%;"> 
בחירת נתיב
<h1 style="text-align:right; font-size:100%;"> 
שמירה כטבלה

In [ ]:
csv_dir = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\csv\SequenceActions.csv'
SequenceActionsTable.to_csv(csv_dir, index=False)

In [ ]:
subprocess.call([r"M:\ncdbScripts\scripts\arcpyPro\SequenceActions-Save as DB Table.bat"]);